In [1]:
# === Standard Library ===
import os
import json
import random
from pathlib import Path
import pandas as pd
# === Third-party Libraries ===
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from tqdm import tqdm
#import albumentations as A
from glob import glob

# === Seaborn Style Configuration ===
sns.set_style("white")
sns.set_palette("gray")  # Grayscale for some journals

# === Dataset Label Mapping ===
DATASET_LABELS = {
    'haldensued08': 'F10',
    'haldennord09': 'F17',
    'haldennord10': 'F2',
    'haldensued10': 'F51',
    'lightly': 'FX'
}

# === Paths and Dataset Selection ===
darwin_path = (
    "/mnt/Data-Work-RE/26_Agricultural_Engineering-RE/263_DP/00_Darwin/digital-production"
)

datasets = [
    'haldennord09',
    'haldensued10',
    'haldensued08',
    'haldennord10',
    'lightly'
]



ModuleNotFoundError: No module named 'seaborn'

In [9]:

def plot_cross_validation_matrix_with_placeholders(df, im_dir, ann_dir, crop_height=678, crop_width=678, output_path='./figures/cv_matrix_placeholder.png'):
    # Select 10 random datasets
    datasets = df['dataset'].unique()
    if len(datasets) < 10:
        raise ValueError("Not enough datasets to select 10 from.")
    selected_datasets = random.sample(list(datasets), 10)

    # Prepare dataset -> {img_path, field_label}
    reference_info = {}
    for dataset in selected_datasets:
        subset = df[df['dataset'] == dataset]
        field_label = subset['field_label'].iloc[0]  # assume consistent label
        img_paths = glob(os.path.join(im_dir, f"{dataset}*.jpg")) + glob(os.path.join(im_dir, f"{dataset}*.png"))
        if not img_paths:
            raise ValueError(f"No images found for dataset {dataset}")
        reference_info[dataset] = {
            'img_path': random.choice(img_paths),
            'field_label': field_label
        }

    fig, axs = plt.subplots(11, 11, figsize=(30, 30))

    # Fill top-left cell with diagonal split
    ax = axs[0, 0]
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.plot([0, 1], [1, 0], color='black', linewidth=2)  # Diagonal
    ax.text(0.95, 0.85, "Model", fontsize=14, ha='right', va='top', fontweight='bold')
    ax.text(0.05, 0.15, "Data", fontsize=14, ha='left', va='bottom', fontweight='bold')
    ax.axis('off')

    # Fill top row with images and field_labels
    for col in range(1, 11):
        dataset = selected_datasets[col - 1]
        img_path = reference_info[dataset]['img_path']
        field_label = reference_info[dataset]['field_label']

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        axs[0, col].imshow(image)
        axs[0, col].set_title(f"{field_label}", fontsize=10)
        axs[0, col].axis('off')

    # Fill left column with images and field_labels
    for row in range(1, 11):
        dataset = selected_datasets[row - 1]
        img_path = reference_info[dataset]['img_path']
        field_label = reference_info[dataset]['field_label']

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        axs[row, 0].imshow(image)
        axs[row, 0].set_ylabel(f"{field_label}", fontsize=10)
        axs[row, 0].axis('off')

    # Fill rest of matrix with placeholder performance values
    for row in range(1, 11):
        for col in range(1, 11):
            ax = axs[row, col]
            ax.axis('off')
            if row != col:
                p1 = round(random.uniform(0.7, 0.9), 2)
                p2 = round(random.uniform(0.7, 0.9), 2)
                p3 = round(random.uniform(0.7, 0.9), 2)
                ax.text(0.5, 0.6, f"P1: {p1}", fontsize=12, ha='center')
                ax.text(0.5, 0.4, f"P2: {p2}", fontsize=12, ha='center')
                ax.text(0.5, 0.2, f"P3: {p3}", fontsize=12, ha='center')
            else:
                ax.text(0.5, 0.5, "X", fontsize=14, ha='center', color='gray')

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300)
    plt.close()
    print(f"Cross-validation matrix saved to {output_path}")


In [10]:
df = pd.read_csv('../assets/lightly_totalimages_selectedimages_annotated_with_fields_label.csv')
im_dir = os.path.join(darwin_path, 'lightly/images')
ann_dir = os.path.join(darwin_path, 'lightly/releases/1/annotations')

plot_cross_validation_matrix_with_placeholders(df, im_dir, ann_dir)

Cross-validation matrix saved to ./figures/cv_matrix_placeholder.png
